In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [9]:
import pandas as pd

df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [11]:
df['bmi'] = df['bmi'].fillna((df['bmi'].median()))
df['smoking_status'] = df['smoking_status'].astype('category')
df['gender'] = df['gender'].astype('category')
df['Residence_type'] = df['Residence_type'].astype('category')
df['work_type'] = df['work_type'].astype('category')
df['ever_married'] = df['ever_married'].astype('category')

In [12]:
df['smoking_status'] = df['smoking_status'].cat.codes
df['gender'] = df['gender'].cat.codes
df['Residence_type'] = df['Residence_type'].cat.codes
df['work_type'] = df['work_type'].cat.codes
df['ever_married'] = df['ever_married'].cat.codes

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   int8   
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   int8   
 6   work_type          5110 non-null   int8   
 7   Residence_type     5110 non-null   int8   
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                5110 non-null   float64
 10  smoking_status     5110 non-null   int8   
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), int8(5)
memory usage: 304.5 KB


In [14]:
df = df.drop(columns = ['Residence_type'], axis = 1)
df = df.drop(columns = ['ever_married'], axis = 1)
df = df.drop(columns = ['id'], axis = 1)

In [15]:
df[:5]

,gender,age,hypertension,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,2,228.69,36.6,1,1
1,0,61.0,0,0,3,202.21,28.1,2,1
2,1,80.0,0,1,2,105.92,32.5,2,1
3,0,49.0,0,0,2,171.23,34.4,3,1
4,0,79.0,1,0,3,174.12,24.0,2,1


In [16]:
df.drop(df[df['gender'] == 'Other'].index, inplace = True)
df.drop(df[df['bmi'] > 47].index, inplace = True)

In [25]:
X = df.iloc[:, 1:7].values
Y = df.iloc[:, 8].values

In [26]:
X

array([[ 67.  ,   0.  ,   1.  ,   2.  , 228.69,  36.6 ],
       [ 61.  ,   0.  ,   0.  ,   3.  , 202.21,  28.1 ],
       [ 80.  ,   0.  ,   1.  ,   2.  , 105.92,  32.5 ],
       ...,
       [ 35.  ,   0.  ,   0.  ,   3.  ,  82.99,  30.6 ],
       [ 51.  ,   0.  ,   0.  ,   2.  , 166.29,  25.6 ],
       [ 44.  ,   0.  ,   0.  ,   0.  ,  85.28,  26.2 ]])

In [61]:
type(Y)

numpy.ndarray

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2, 
                                                    random_state=4)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3994, 6), (999, 6), (3994,), (999,))

In [46]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [55]:
from sklearn.utils import resample

train = {}
train['data'] = X_train.copy()
train['targ'] = y_train

train

{'data': array([[ 45.  ,   0.  ,   0.  ,   2.  ,  64.14,  29.4 ],
        [ 78.  ,   0.  ,   0.  ,   2.  , 100.09,  30.5 ],
        [ 10.  ,   0.  ,   0.  ,   4.  ,  95.8 ,  17.3 ],
        ...,
        [ 45.  ,   0.  ,   0.  ,   2.  , 106.83,  32.1 ],
        [ 78.  ,   1.  ,   0.  ,   3.  ,  56.11,  25.5 ],
        [ 35.  ,   0.  ,   0.  ,   0.  ,  82.39,  33.2 ]]),
 'targ': array([1, 0, 0, ..., 0, 1, 0])}

In [59]:
stroke = []
n_stroke = []

for i in range(len(train['data'])):
    if train['targ'][i] == 1:
        stroke.append(train['data'][i])
    else:
        n_stroke.append(train['data'][i])
        
len(stroke), len(n_stroke)

(196, 3798)

In [60]:
import random

new_n_stroke = []
for i in range(len(stroke)):
    idx = random.randrange(0, len(n_stroke))
    new_n_stroke.append(n_stroke[idx])
    n_stroke.pop(idx)

In [62]:
targets = []
for i in range(len(stroke)):
    targets.append(1)
for i in range(len(new_n_stroke)):
    targets.append(0)

In [63]:
len(stroke), len(new_n_stroke)

(196, 196)

In [64]:
from sklearn.utils import shuffle

new_X = np.array(stroke + new_n_stroke)
targets = np.array(targets)

X_train, y_train = shuffle(new_X, targets)

In [65]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def metrics(y, y_pred):
    print("MSE: ", mean_squared_error(y, y_pred))
    print("MAE: ", mean_absolute_error(y, y_pred))
    print("RMS: ", mean_squared_error(y, y_pred, squared=False))

# Линейная регрессия

In [66]:
class LinearRegression_(BaseEstimator, ClassifierMixin):
    def __init__(self, lr = 1, epoch = 5):
        self.lr = lr
        self.epoch = epoch
        self.m = None
                
    def loss(self, y_pred, y):
        return np.sum(np.square(y_pred-y))/(2*self.m)
            
    def fit(self, X, y):
        self.w = np.zeros(X.shape[1])
        self.b = 0
        self.m = X.shape[0]   
        self.mean, self.std = X.mean(axis=0), X.std(axis=0)
        
        for i in range(self.epoch):
            
            y_pred = np.dot(X,self.w)
            loss = self.loss(y_pred, y)
            
            h = np.dot(X, self.w)+self.b
            dw = np.dot(X.T,(h-y)) / self.m
            db = np.sum(h-y)  / self.m
            self.w = self.w - self.lr*dw
            self.b = self.b - self.lr*db
        
    def normalizeX(self,X):
        return (X-self.mean) / self.std
        
    def predict(self, X):
        X = self.normalizeX(X)
        return np.dot(X,self.w)+self.b
    
    def transform(self, X):
        return X

In [67]:
pipe = Pipeline(steps=[('lin', LinearRegression_())])
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'lin', 'lin__epoch', 'lin__lr'])

In [68]:
parameters_grid = {
    'lin__epoch': [1, 2, 5, 7, 10],
    'lin__lr': [0.001, 0.01, 0.1, 0.5, 1],
}

grid_cv = GridSearchCV(pipe, parameters_grid,scoring = 'neg_mean_squared_error')
grid_cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('lin', LinearRegression_())]),
             param_grid={'lin__epoch': [1, 2, 5, 7, 10],
                         'lin__lr': [0.001, 0.01, 0.1, 0.5, 1]},
             scoring='neg_mean_squared_error')

In [69]:
grid_cv.best_params_

{'lin__epoch': 1, 'lin__lr': 0.001}

In [70]:
grid_cv.best_score_

-0.4711384695924309

In [71]:
import joblib
joblib.dump(grid_cv.best_estimator_, "lin.plk")

['lin.plk']

In [72]:
grid_cv.transform(X_train)

array([[ 64.  ,   0.  ,   1.  ,   0.  ,  74.1 ,  28.8 ],
       [ 63.  ,   0.  ,   0.  ,   2.  , 228.56,  27.4 ],
       [ 54.  ,   0.  ,   0.  ,   3.  , 191.82,  40.4 ],
       ...,
       [ 66.  ,   0.  ,   0.  ,   2.  , 190.4 ,  28.1 ],
       [ 65.  ,   0.  ,   0.  ,   3.  ,  68.43,  28.1 ],
       [ 59.  ,   0.  ,   0.  ,   2.  ,  96.16,  44.1 ]])

In [73]:
y_pred = grid_cv.predict(X_test)

In [74]:
metrics(y_test,y_pred)

MSE:  0.05423808919263267
MAE:  0.11526361750589442
RMS:  0.23289072371529243


In [75]:
from sklearn.linear_model import LinearRegression

lin = LinearRegression()
lin.fit(X_train, y_train)
metrics(y_test,lin.predict(X_test))

MSE:  0.17938580410193433
MAE:  0.34153179489839997
RMS:  0.4235396133798282


# Метод опорных векторов

In [76]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

In [77]:
def rbf(x, y, gamma):
    return np.exp(-gamma*np.sum((y - x[:,np.newaxis])**2, axis=-1))

def linear(x, y, gamma):
    return np.dot(x, y.T)

In [78]:
class SVMM(BaseEstimator, ClassifierMixin):
    def __init__(self, lr=1, epoch=1000):
        self.lr = lr        
        self.epoch = epoch

    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        y_ = np.where(y > 0, 1, -1)
        
        self.w = np.zeros(n_features)

        for e in range(self.epoch):
            for i, x in enumerate(X):
                a = y_[i] * (np.dot(x, self.w))
                if a < 1:
                    self.w += self.lr * (X[i]*y_[i] - 2/self.epoch*self.w)
                else:
                    self.w += self.lr * (-2/self.epoch*self.w)


    def predict(self, X):
        y_pred = np.dot(X, self.w)
        return np.where(y_pred > 0, 1, 0)
    
    def transform(self, X):
        return X

In [79]:
pipe = Pipeline(steps=[('svm', SVMM())])
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'svm', 'svm__epoch', 'svm__lr'])

In [80]:
parameters_grid = {
    'svm__lr': [0.001, 0.1, 0.5, 1],    
    'svm__epoch': [20, 50, 100, 1000, 5000]    
}

grid_cv = GridSearchCV(pipe, parameters_grid)
grid_cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('svm', SVMM())]),
             param_grid={'svm__epoch': [20, 50, 100, 1000, 5000],
                         'svm__lr': [0.001, 0.1, 0.5, 1]})

In [81]:
grid_cv.best_params_

{'svm__epoch': 20, 'svm__lr': 0.001}

In [82]:
grid_cv.best_score_

0.6556312885426809

In [83]:
joblib.dump(grid_cv.best_estimator_, "svm.plk")

['svm.plk']

In [84]:
grid_cv.transform(X_train);
y_pred = grid_cv.predict(X_test)
metrics(y_test,y_pred)

MSE:  0.11511511511511512
MAE:  0.11511511511511512
RMS:  0.33928618468059546


In [85]:
print("Confusion matrix", confusion_matrix(y_test, y_pred))
print("\n\nRecall score", recall_score(y_test, y_pred))
print("\n\nroc_auc score", roc_auc_score(y_test, y_pred))
print("\n\nAccuracy score", accuracy_score(y_test, y_pred))

Confusion matrix [[860  89]
 [ 26  24]]


Recall score 0.48


roc_auc score 0.6931085353003161


Accuracy score 0.8848848848848849


In [86]:
from sklearn.svm import SVC

pipe1 = Pipeline(steps=[('svm', SVC())])
pipe1.fit(X_train, y_train)

Pipeline(steps=[('svm', SVC())])

In [87]:
y_pred = pipe1.predict(X_test)
metrics(y_test,y_pred)

MSE:  0.32232232232232233
MAE:  0.32232232232232233
RMS:  0.5677343765550245


In [88]:
print("Confusion matrix", confusion_matrix(y_test, y_pred))
print("\n\nRecall score", recall_score(y_test, y_pred))
print("\n\nroc_auc score", roc_auc_score(y_test, y_pred))
print("\n\nAccuracy score", accuracy_score(y_test, y_pred))

Confusion matrix [[633 316]
 [  6  44]]


Recall score 0.88


roc_auc score 0.7735089567966281


Accuracy score 0.6776776776776777


# KNN

In [89]:
def euclidean_distance(v1, v2):
    return np.sqrt(np.sum((v1-v2)**2))

def absolute_distance(v1, v2):
    return np.sum(np.absolute(v1-v2))

In [90]:
from scipy.stats import mode

class KNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k = 4, distance_func = euclidean_distance):
        self.k = k
        self.distance_func = distance_func
        
    def fit(self, X, y):
        self.X = X
        self.y = y
        
    def predict(self, x):
        op_labels = []
    
        for item in x: 
            point_dist = []
            for j in range(len(self.X)): 
                distances = self.distance_func(np.array(self.X[j,:]) , item) 
                point_dist.append(distances) 
            
            point_dist = np.array(point_dist) 
            dist = np.argsort(point_dist)[:self.k] 
            labels = self.y[dist]

            lab = mode(labels) 
            lab = lab.mode[0]
            op_labels.append(lab)

        return op_labels
    
    def transform(self, X):
        return X

In [91]:
pipe = Pipeline(steps=[('knn', KNN())])
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'knn', 'knn__distance_func', 'knn__k'])

In [92]:
parameters_grid = {
    'knn__k': [2, 5, 7, 10],
    'knn__distance_func': [euclidean_distance, absolute_distance]    
}

grid_cv = GridSearchCV(pipe, parameters_grid)
grid_cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('knn', KNN())]),
             param_grid={'knn__distance_func': [<function euclidean_distance at 0x7f2549b05700>,
                                                <function absolute_distance at 0x7f2549b05430>],
                         'knn__k': [2, 5, 7, 10]})

In [93]:
grid_cv.best_params_

{'knn__distance_func': <function __main__.absolute_distance(v1, v2)>,
 'knn__k': 7}

In [94]:
grid_cv.best_score_

0.7271340473872119

In [95]:
joblib.dump(grid_cv.best_estimator_, "knn.plk")

['knn.plk']

In [96]:
grid_cv.transform(X_train);
y_pred = grid_cv.predict(X_test)
metrics(y_test,y_pred)

MSE:  0.33933933933933935
MAE:  0.33933933933933935
RMS:  0.5825284021739535


In [97]:
print("Confusion matrix\n", confusion_matrix(y_test, y_pred))
print("\n\nRecall score", recall_score(y_test, y_pred))
print("\n\nroc_auc score", roc_auc_score(y_test, y_pred))
print("\n\nAccuracy score", accuracy_score(y_test, y_pred))

Confusion matrix
 [[619 330]
 [  9  41]]


Recall score 0.82


roc_auc score 0.7361327713382507


Accuracy score 0.6606606606606606


In [98]:
from sklearn.neighbors import KNeighborsClassifier

pipe1 = Pipeline(steps=[('knn_', KNeighborsClassifier(n_neighbors=10))])
pipe1.fit(X_train, y_train)
y_pred = pipe1.predict(X_test)
metrics(y_test,y_pred)

print("Confusion matrix\n", confusion_matrix(y_test, y_pred))
print("\n\nRecall score", recall_score(y_test, y_pred))
print("\n\nroc_auc score", roc_auc_score(y_test, y_pred))
print("\n\nAccuracy score", accuracy_score(y_test, y_pred))

MSE:  0.3203203203203203
MAE:  0.3203203203203203
RMS:  0.5659684799706785
Confusion matrix
 [[640 309]
 [ 11  39]]


Recall score 0.78


roc_auc score 0.7271970495258167


Accuracy score 0.6796796796796797


# Наивный баессовский классификатор

In [105]:
class Bayes(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass

    def separate(self, X, y):
        classes = {}
        for i in range(len(X)):
            if y[i] not in classes:
                classes[y[i]] = []
            classes[y[i]].append(X[i])
        return classes

    def mean_std(self, X):
        for i in zip(*X):
            yield {
                'std' : np.std(i), 'mean' : np.mean(i)
            }


    def fit (self, X, y):
        sep_classes = self.separate(X, y)
        self.class_summary = {}

        for i, j in sep_classes.items():
            self.class_summary[i] = {
                'prior': len(j)/len(X),
                'summ': [k for k in self.mean_std(j)],
            }

    def gaus_distribution(self, x, mean, std):
        return np.exp(-((x-mean)**2 / (2*std**2))) / (np.sqrt(2*np.pi)*std)


    def predict(self, X):        
        max_apr = []

        for row in X:
            proba = {}
            
            for cl_name, ftrs in self.class_summary.items():
                ttl_ftrs =  len(ftrs['summ'])
                h = 1
                for idx in range(ttl_ftrs):
                    tmp = row[idx]
                    mean = ftrs['summ'][idx]['mean']
                    stdev = ftrs['summ'][idx]['std']
                    gaus_proba = self.gaus_distribution(tmp, mean, stdev)
                    h *= gaus_proba
                proba[cl_name] = ftrs['prior'] * h

            MAP = max(proba, key = proba.get)
            max_apr.append(MAP)

        return max_apr
    
    def transform(self, X):
        return X

In [106]:
pipe = Pipeline(steps=[('bayes', Bayes())])
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'bayes'])

In [107]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
metrics(y_pred, y_test)

MSE:  0.18618618618618618
MAE:  0.18618618618618618
RMS:  0.431492973507317


In [108]:
print("Confusion matrix\n", confusion_matrix(y_test, y_pred))
print("\n\nRecall score", recall_score(y_test, y_pred))
print("\n\nroc_auc score", roc_auc_score(y_test, y_pred))
print("\n\nAccuracy score", accuracy_score(y_test, y_pred))

Confusion matrix
 [[788 161]
 [ 25  25]]


Recall score 0.5


roc_auc score 0.6651738672286617


Accuracy score 0.8138138138138138


In [109]:
joblib.dump(pipe, "nb.plk")

['nb.plk']

In [110]:
from sklearn.naive_bayes import GaussianNB

pipe1 = Pipeline(steps=[('nb', GaussianNB())])
pipe1.fit(X_train, y_train)
y_pred = pipe1.predict(X_test)
metrics(y_test,y_pred)

print("Confusion matrix\n", confusion_matrix(y_test, y_pred))
print("\n\nRecall score", recall_score(y_test, y_pred))
print("\n\nroc_auc score", roc_auc_score(y_test, y_pred))
print("\n\nAccuracy score", accuracy_score(y_test, y_pred))

MSE:  0.18618618618618618
MAE:  0.18618618618618618
RMS:  0.431492973507317
Confusion matrix
 [[788 161]
 [ 25  25]]


Recall score 0.5


roc_auc score 0.6651738672286617


Accuracy score 0.8138138138138138
